In [1]:
import numpy as np
import pandas as pd

import sys, os, yaml, ssl, asyncio



# mapping to the modules that make the app
sys.path.insert(0, "../..")
# sys.path.insert(0, "..")


In [2]:
from app.objects import structures

from notebooks.helpers import test_queries

In [6]:
ssl._create_default_https_context = ssl._create_unverified_context
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
import nest_asyncio

# this is required for running in a Jupyter Notebook.
nest_asyncio.apply()

import app.models as models
import app.connectors.cmdb_graph as cmdb_graph

c = cmdb_graph.CosmosdbClient()

executing local windows deployment


In [7]:
buildings = yaml.safe_load(
    open(os.path.join(os.getenv("abspath"), "app", "configurations", "buildings.yaml"))
)
buildings["explanation"]

[{'type': 'id_name of the building. lower, underscores'},
 {'label': "'class' of the object. in this case it is always 'building'"},
 {'name': "pretty printed name of the building. UI can remove the '_' at display time."},
 {'augments_resource': 'OPTIONAL augments a specific resource, creates it if it does not exist'},
 {'owned_by': 'type of object that can own/maintain this building, assuming a relationhsip to that object'},
 {'faction_augments': 'if owned by a pop in the faction,  raises the stat for all pops in factions'},
 {'effort': 'time in universal time units (UTU) that the action will take to complete'},
 {'requires_attr': 'required attributes of the agent. Determines if the agent can do the action.'},
 {'requires_faction': 'requires that the faction have this attribute'},
 {'render_type': "guides the rendering in buildings.js. Available types: 'block'"},
 {'renews_faction_resource': 'augments (or diminishes) a resource in the faction. Creates if it does not exist.'},
 {'renew

# Creating a new Building

In [8]:
userguid = "8d5b667f-b225-4641-b499-73b77558ff86"
pop = test_queries.get_random_pop(c,userguid)
pop


{'isIdle': 'false',
 'name': 'Hiborsouth Tsufen',
 'objid': '4407162012340',
 'conformity': '0.344',
 'literacy': '0.394',
 'aggression': '0.445',
 'constitution': '0.408',
 'health': '0.7',
 'isIn': '9086589813558',
 'industry': '0.4265',
 'wealth': '0.41025',
 'factionLoyalty': '0.357',
 'userguid': '8d5b667f-b225-4641-b499-73b77558ff86',
 'objtype': 'pop',
 'id': '4407162012340'}

In [9]:
buildings_list = list(buildings["buildings"].keys())
buildings_list

['farm',
 'hospital',
 'solar_panel',
 'forestry',
 'infrastructure',
 'university',
 'observatory',
 'prospector',
 'oil_well',
 'marketplace',
 'shipyard',
 'flight_control',
 'Launch Complex']

Should be able to search the building type by name

In [14]:
b = buildings["buildings"]['farm']
b

{'type': 'farmland',
 'label': 'building',
 'name': 'Farmland',
 'description': 'Generates organic foodstuffs, populations will consume food before consuming natural resources',
 'planet_requirements': {'isHabitable': 'true'},
 'faction_augments': {'wealth': -1},
 'renews_faction_resource': {'granulated_hydrocarbons': 10},
 'owned_by': 'pop',
 'effort': 1,
 'render_type': 'block'}

In [15]:
building = structures.Building(pop,b)
building

<building: None; 5051465602954; Farmland>

In [16]:
building.get_data()

{'name': 'Farmland',
 'objid': '5051465602954',
 'label': 'building',
 'ownedBy': '4407162012340',
 'type': 'farmland',
 'description': 'Generates organic foodstuffs, populations will consume food before consuming natural resources',
 'planet_requirements': {'isHabitable': 'true'},
 'faction_augments': {'wealth': -1},
 'renews_faction_resource': {'granulated_hydrocarbons': 10},
 'owned_by': 'pop',
 'effort': 1,
 'render_type': 'block'}

There should be a building action. 

In [24]:
c.run_query("g.V().has('label','action').valueMap()")
action = c.clean_nodes(c.res)[0]
action

{'type': 'construction',
 'comment': 'constructing a Farmland',
 'effort': 1,
 'applies_to': 'pop',
 'owned_by': 'pop',
 'building': 'farmland',
 'created_at': 45184,
 'to_build': '{type: farmland, label: building, name: Farmland, description: Generates organic foodstuffs, populations will consume food before consuming natural resources, planet_requirements: {isHabitable: true}, faction_augments: {wealth: -1}, renews_faction_resource: {granulated_hydrocarbons: 10}, owned_by: pop, effort: 1, render_type: block}',
 'objid': '2778015807971',
 'userguid': '8d5b667f-b225-4641-b499-73b77558ff86',
 'objtype': 'action',
 'id': '2778015807971'}

In [25]:
to_build = action['to_build']
to_build

'{type: farmland, label: building, name: Farmland, description: Generates organic foodstuffs, populations will consume food before consuming natural resources, planet_requirements: {isHabitable: true}, faction_augments: {wealth: -1}, renews_faction_resource: {granulated_hydrocarbons: 10}, owned_by: pop, effort: 1, render_type: block}'

In [17]:
building.get_owned_by()

{'node1': '4407162012340', 'node2': '5051465602954', 'label': 'owns'}

In [27]:
data = {'nodes':[building.get_data()],'edges':[building.get_owned_by()]}
c.upload_data(userguid,data)

In [28]:
building_query = f"""
g.V().has('objid','{pop['objid']}').as('pop')
    .out('owns').as('building')
    .path()
        .by(valueMap('objid','name'))
        .by(valueMap())
"""

c.run_query(building_query)
c.res

[{'labels': [['pop'], ['building']],
  'objects': [{'objid': ['2279331817806'], 'name': ['Dorkaitade Newjo']},
   {'name': ['Farmland'],
    'objid': ['2465267355835'],
    'description': ['Generates organic foodstuffs, populations will consume food before consuming natural resources'],
    'planet_requirements': ['{isHabitable: true}'],
    'faction_augments': ['{wealth: -1}'],
    'renews_faction_resource': ['{granulated_hydrocarbons: 10}'],
    'owned_by': ['pop'],
    'effort': [1],
    'render_type': ['block'],
    'userguid': ['8d5b667f-b225-4641-b499-73b77558ff86'],
    'objtype': ['building']}]},
 {'labels': [['pop'], ['building']],
  'objects': [{'objid': ['2279331817806'], 'name': ['Dorkaitade Newjo']},
   {'name': ['Farmland'],
    'objid': ['6203641029822'],
    'description': ['Generates organic foodstuffs, populations will consume food before consuming natural resources'],
    'planet_requirements': ['{isHabitable: true}'],
    'faction_augments': ['{wealth: -1}'],
    'r

# The Building and Faction resolver



In [48]:
def get_faction_pop_structures(c):
    building_query = f"""
    g.V().has('label','faction').as('faction').in('isIn').has('label','pop').as('pop').out('owns').as('structure').path().by(valueMap())
    """
    c.run_query(building_query)
    faction_res = c.query_to_dict(c.res)
    print(f"EXOADMIN: number of items: {len(faction_res)}")
    for item in faction_res:
        item['action'] = 'structure'
    return faction_res

get_faction_pop_structures(c)

EXOADMIN: number of items: 3


[{'faction': {'name': 'Dorkaitade',
   'objid': '5893520393492',
   'lat': '0.097',
   'long': '-0.0',
   'userguid': '8d5b667f-b225-4641-b499-73b77558ff86',
   'objtype': 'faction',
   'id': '5893520393492'},
  'pop': {'name': 'Dorkaitade Newjo',
   'objid': '2279331817806',
   'conformity': '0.431',
   'literacy': '0.425',
   'aggression': '0.454',
   'constitution': '0.597',
   'health': '0.7',
   'isIn': '5893520393492',
   'industry': '0.5255',
   'wealth': '0.47524999999999995',
   'factionLoyalty': '0.622',
   'isIdle': 'true',
   'userguid': '8d5b667f-b225-4641-b499-73b77558ff86',
   'objtype': 'pop',
   'id': '2279331817806'},
  'structure': {'name': 'Farmland',
   'objid': '2465267355835',
   'description': 'Generates organic foodstuffs, populations will consume food before consuming natural resources',
   'planet_requirements': '{isHabitable: true}',
   'faction_augments': '{wealth: -1}',
   'renews_faction_resource': '{granulated_hydrocarbons: 10}',
   'owned_by': 'pop',
  

First getting a list of all of the factions, that have pops, that have buildings. 

In [31]:
building_query = f"""
g.V().has('label','faction').as('faction').in('isIn').has('label','pop').as('pop').out('owns').as('structure').path().by(valueMap())
"""

c.run_query(building_query)
faction_res = c.query_to_dict(c.res)
print(f"number of items: {len(faction_res)}")
faction_res

number of items: 3


[{'faction': {'name': 'Dorkaitade',
   'objid': '5893520393492',
   'lat': '0.097',
   'long': '-0.0',
   'userguid': '8d5b667f-b225-4641-b499-73b77558ff86',
   'objtype': 'faction',
   'id': '5893520393492'},
  'pop': {'name': 'Dorkaitade Newjo',
   'objid': '2279331817806',
   'conformity': '0.431',
   'literacy': '0.425',
   'aggression': '0.454',
   'constitution': '0.597',
   'health': '0.7',
   'isIn': '5893520393492',
   'industry': '0.5255',
   'wealth': '0.47524999999999995',
   'factionLoyalty': '0.622',
   'isIdle': 'true',
   'userguid': '8d5b667f-b225-4641-b499-73b77558ff86',
   'objtype': 'pop',
   'id': '2279331817806'},
  'structure': {'name': 'Farmland',
   'objid': '2465267355835',
   'description': 'Generates organic foodstuffs, populations will consume food before consuming natural resources',
   'planet_requirements': '{isHabitable: true}',
   'faction_augments': '{wealth: -1}',
   'renews_faction_resource': '{granulated_hydrocarbons: 10}',
   'owned_by': 'pop',
  

The function will need to create event messges for these steps:
* renews_faction_resource

## Renews_Faction_Resource

In [46]:
def renew_create_faction_resource(c,faction,resource):
    faction_has_resource_query = f"""
    g.V().has('objid','{faction['objid']}').out('has').has('label','resource').has('objid','{resource[0]}').count()
    """
    c.run_query(faction_has_resource_query)
    r = c.res
    if r[0] == 0:
        create_faction_resource_query = f"""
        g.V().has('objid','{resource[0]}')
        """
        c.run_query(create_faction_resource_query)
        r = c.res
    return r


# faction set is one of the faction,pop,structure combos. 
faction_set = faction_res[0]

# renew_create_faction_resource(c,faction_set['faction'],faction_set['structure'])

if faction_set['structure'].get('renews_faction_resource','does not renew resources') != 'does not renew resources':
    resources = yaml.safe_load(faction_set['structure'].get('renews_faction_resource'))
    for resource in list(resources.items()):
        r = renew_create_faction_resource(c,faction_set['faction'],resource)
        print(r)



GremlinServerError: 597: 

ActivityId : 3714d0ab-508b-4545-a564-9589385afdff
ExceptionType : GraphCompileException
ExceptionMessage : 
	Gremlin Query Compilation Error: Unable to find any method 'next' @ line 2, column 109.
	1 Error(s)
	GremlinRequestId : 967acf69-2ed5-4f93-af12-9d580bcba8f2
	Context : graphcompute
	Scope : graphparse-translate-csharpexpressionbinding
	GraphInterOpStatusCode : QuerySyntaxError
	HResult : 0x80131500


In [44]:
i[0]

'granulated_hydrocarbons'